In [1]:
import theano
import theano.tensor as T
from theano.tensor.signal import pool

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class cnn_layer():
    def __init__(self, output_activation_func=T.nnet.relu, K_init, b_init, pool_size=(2,2)):
        '''
        K_init: Initial values for the kernel for this layer
        b_init: Initial values for the bias for this layer
        activation: RelU
        pool_size: size over which max_pool is applied
        '''
        
        self.K = theano.shared(value=K_init.astype(theano.config.floatX),
                               borrow=True
                              )
        self.b = theano.shared(value=b_init.astype(theano.config.floatX),
                               borrow=True
                              )
        
        
        self.params = [self.K, self.b]
    
    def output(self, I):
        assert I.shape[1] == K_init.shape[1]
        self.I = I
        
        conv_out = T.nnet.conv2d(I, self.K)
        pool_out = pool.pool_2d(conv_out, pool_size, ignore_border=True)
        
        return(output_activation_func(pool_out + self.b.dimshuffle('x', 0, 'x', 'x')))

In [ ]:
class CNN():
    def __init__(self, n_layer, output_activation_func, K_init, b_init):
        '''
        output of sequential layers

        n_layer: number of convolutional layers
        '''
        self.layers = [cnn_layer(K_init, b_init) for i in range(n_layer-1)]
        self.layers.append(cnn_layer(output_activation_func, K_init, b_init))
            
        self.params = [item for layer in self.layers for item in layer.params]
        #mlp.params is a list of parameters, not a list of lists of parameters
    
    def output(self, X):
        for layer in self.layers:
            X = layer.output(X)
        return(X)
    
    def error(self, X, y):
        return(T.sum((y-self.output(X))**2))